# Nuclio - Training function

## Environment

In [147]:
# nuclio: ignore
import nuclio

### Configurations

In [148]:
%%nuclio config

# Trigger
spec.triggers.retrain.kind = "cron"
spec.triggers.retrain.attributes.interval = "1h"

# Base image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.retrain.kind to 'cron'
%nuclio: setting spec.triggers.retrain.attributes.interval to '1h'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Variables

In [149]:
%%nuclio env

# Work from TSDB or Parquet?
FROM_TSDB=0

# DB Config
V3IO_FRAMESD=${V3IO_FRAMESD}
V3IO_USERNAME=${V3IO_USERNAME}
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Features
FEATURES_TABLE=../netops_features
# FEATURES_TABLE=/v3io/bigdata/netops_features_parquet

# Predictions
PREDICTIONS_TABLE=../netops_predictions
# PREDICTIONS_TABLE=/v3io/bigdata/netops_predictions_parquet

# Training
TRAIN_ON_LAST=1d
TRAIN_SIZE=0.7

# Parallelizem
NUMBER_OF_SHARDS=4

# Model
# MODEL_FILENAME=netops.model.pickle
MODEL_FILENAME=netops.v3.model.pickle
SOURCE_MODEL_DIR=../models #/bigdata/netops/models
FIXED_WEB_DIR=/models

%nuclio: setting 'FROM_TSDB' environment variable
%nuclio: setting 'V3IO_FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'FEATURES_TABLE' environment variable
%nuclio: setting '# FEATURES_TABLE' environment variable
%nuclio: setting 'PREDICTIONS_TABLE' environment variable
%nuclio: setting '# PREDICTIONS_TABLE' environment variable
%nuclio: setting 'TRAIN_ON_LAST' environment variable
%nuclio: setting 'TRAIN_SIZE' environment variable
%nuclio: setting 'NUMBER_OF_SHARDS' environment variable
%nuclio: setting '# MODEL_FILENAME' environment variable
%nuclio: setting 'MODEL_FILENAME' environment variable
%nuclio: setting 'SOURCE_MODEL_DIR' environment variable
%nuclio: setting 'FIXED_WEB_DIR' environment variable


%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line


In [150]:
%nuclio env -c CURRENT_MODEL_DIR=/models
%nuclio env -l CURRENT_MODEL_DIR=../models

%nuclio: setting 'CURRENT_MODEL_DIR' environment variable


### Commands

In [151]:
%%nuclio cmd -c

############
# installs #
############

# Utils
pip install pyarrow
pip install pandas

# Igz DB
pip install v3io_frames --upgrade

# Function
pip install xgboost
pip install scikit-learn==0.20.1

apt-get update && apt-get install -y wget
mkdir -p ${FIXED_WEB_DIR}

In [152]:
%%nuclio cmd -c 
# Copy the model file into the function
wget -O ${FIXED_WEB_DIR}/${MODEL_FILENAME} --header "x-v3io-session-key: ${V3IO_ACCESS_KEY}" http://${V3IO_WEBAPI_SERVICE_HOST}:8081${SOURCE_MODEL_DIR}/${MODEL_FILENAME}

## Function

### Imports

In [234]:
import os
import pickle

import v3io_frames as v3f

from datetime import datetime
import pandas as pd
import cudf
import xgboost as xgb

### Helper functions

In [154]:
def format_df_from_tsdb(context, df):
    df.index.names = ['timestamp', 'company', 'data_center', 'device']
    return df

In [155]:
def set_indexes(df):
    df = df.set_index(['timestamp', 'company', 'data_center', 'device'])
    return df

In [156]:
def get_data_tsdb(context):
    df = context.v3f.read(backend='tsdb', query=f'select * from {context.features_table}',
                          start=f'now-{context.train_on_last}', end='now', multi_index=True)
    df = format_df_from_tsdb(context, df)
    
    # Keep columns
    keep_columns = [col for col in df.columns if 'is_error' not in col]
    
    # Keep good columns and Sort them
    df = df[sorted(keep_columns)]
    
    return df

In [157]:
def get_data_parquet(context):
    # Get parquet files
    mpath = [os.path.join(context.features_table, file) for file in os.listdir(context.features_table)]
    
    # Get latest filename
    latest = max(mpath, key=os.path.getmtime)
    print(latest)
    context.logger.debug(f'Reading data from: {latest}')
    
    # Load parquet to dask
    df = cudf.read_parquet(latest)
    
    # Keep columns
    keep_columns = [col for col in df.columns if 'is_error' not in col]
    
    # Keep good columns and Sort them
    df = df[sorted(keep_columns)]
    
    return df

In [158]:
def save_to_tsdb(context, df: pd.DataFrame):   
    # Fix indexes before saving to TSDB
    df = set_indexes(df)
    
    # Save to TSDB
    context.v3f.write('tsdb', context.predictions_table, df)

In [312]:
def save_to_parquet(context, df: pd.DataFrame):
    print('Saving features to Parquet')
    
    # Save parquet
    first_timestamp = pd.to_datetime(df.head(1).to_pandas()['timestamp'].values[0]).strftime('%Y%m%dT%H%M%S')
    last_timestamp = pd.to_datetime(df.tail(1).to_pandas()['timestamp'].values[0]).strftime('%Y%m%dT%H%M%S')
    filename = first_timestamp + '-' + last_timestamp + '.parquet'
    filepath = os.path.join(context.predictions_table, filename)

    df.to_parquet(filepath)

### Init context

In [313]:
def init_context(context):
    
    # Save features directory
    features_table = os.getenv('FEATURES_TABLE', 'netops_features')
    setattr(context, 'features_table', features_table)
    
    # Save predictions directory
    predictions_table = os.getenv('PREDICTIONS_TABLE', 'netops_predictions')
    setattr(context, 'predictions_table', predictions_table)
    
    # Get saving configuration
    is_from_tsdb = (int(os.getenv('FROM_TSDB', 1)) == 1)
    
    # Save to TSDB
    if is_from_tsdb:
        # Create our DB client
        v3io_client = v3f.Client(address='http://' + os.getenv('V3IO_FRAMESD', 'framesd:8081'), 
                            container='bigdata', 
                            password=os.environ['V3IO_ACCESS_KEY'], 
                            user=os.environ['V3IO_USERNAME'])
        setattr(context, 'v3f', v3io_client)
        
        # Create predictions table if neede
        context.v3f.create('tsdb', context.predictions_table, attrs={'rate': '1/s'}, if_exists=1)
        
        train_on_last = os.getenv('TRAIN_ON_LAST', '1h')
        setattr(context, 'train_on_last', train_on_last)
        
        # Set TSDB reading function
        setattr(context, 'read', get_data_tsdb)
        
        # Set TSDB saving fucntion
        setattr(context, 'write', save_to_tsdb)
        
    # Save to Parquet
    else:
         # Create saving directory if needed
        filepath = os.path.join(context.predictions_table)
        if not os.path.exists(filepath):
            os.makedirs(filepath)
            
        # Set Parquet reading function
        setattr(context, 'read', get_data_parquet)
        
        # Set Parquet saving fucntion
        setattr(context, 'write', save_to_parquet)
    
    # Load the model
    model_path = os.path.join(os.getenv('CURRENT_MODEL_DIR', '/models'), os.getenv('MODEL_FILENAME', 'netops.v1.model'))
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    setattr(context, 'model', model)

### Handler

In [314]:
def handler(context, event):

    # Load last hour data
    df = context.read(context)
    
    # limit for testing
    df = df.head(2)
    X = df.drop(['timestamp', 'company', 'data_center', 'device'])
    X = xgb.DMatrix(data=X.as_gpu_matrix())
    
    # Predict
    df['prediction'] = context.model.predict(X)
    
    # Save
    context.write(context, df)

## Test

In [315]:
# nuclio: ignore
init_context(context)

In [319]:
# nuclio: ignore
event = nuclio.Event(body='')
output = handler(context, event)
output

../netops_features/20190606T080717-20190607T080527.parquet
Saving features to Parquet


## Deployment

In [176]:
%nuclio deploy -p netops -n predict -c

%nuclio: ['deploy', '-p', 'netops', '-n', 'predict', '-c', '/User/netops/tutorials/demos/netops/Nuclio-Inference.ipynb']
%nuclio: [nuclio.deploy] 2019-05-20 09:18:24,965 (info) Building processor image
%nuclio: [nuclio.deploy] 2019-05-20 09:18:31,035 (info) Pushing image
%nuclio: [nuclio.deploy] 2019-05-20 09:18:31,035 (info) Build complete
%nuclio: [nuclio.deploy] 2019-05-20 09:18:37,109 (info) Function deploy complete
%nuclio: [nuclio.deploy] 2019-05-20 09:18:37,116 done updating predict, function address: 3.122.56.83:32642
%nuclio: function deployed


In [303]:
# nuclio: ignore
pd.read_parquet('../netops_predictions/20190606T080717-20190606T080717.parquet/5e7106503a0f42b686f81d3da4b47f0b.parquet')

,company,cpu_utilization_hourly,cpu_utilization_minute,cpu_utilization_raw,data_center,device,latency_hourly,latency_minute,latency_raw,packet_loss_hourly,packet_loss_minute,packet_loss_raw,throughput_hourly,throughput_minute,throughput_raw,timestamp,prediction
0,Wade-Burton,70.500865,73.070356,63.856445,Ibarra_Motorway,7173220258994,1.858491,1.059785,0.000000,0.782009,0.90359,0.0,250.769755,252.908534,252.848657,2019-06-06,0.309667
1,Harris-Klein,70.495638,72.970066,80.825313,Kennedy_Oval,1814628377378,1.884096,2.596091,4.608918,0.779490,0.90359,0.0,250.623596,239.929399,221.674161,2019-06-06,0.977703
